In [2]:
import gate
import numpy as np

init = [1, 0, 0, 0]
print(gate.gate2['CX'] @ (np.kron(gate.gate1['H'],gate.gate1['I']) @ init))

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [10]:
for i in range(0, 10):
    k = np.kron(gate.gate1['I'],gate.gate1['H'])


In [ ]:
for i in range(0, 1000000):
    k = gate.gate2['IH']

In [27]:
def kron_A_I(A, N):
    out = np.zeros((N*N, N*N),dtype=A.dtype)
    for i in range(N):
        for j in range(N):
            value = A[i,j]
            out[i*2: i*2 + 2,j*2:j*2 + 2] = np.array([[value, 0], [0, value]])
    return out

In [68]:
A = np.random.rand(4, 4)
print(A)

[[0.08438087 0.41107181 0.04538587 0.95188399]
 [0.1705832  0.77136947 0.69109793 0.58784028]
 [0.29480806 0.77784422 0.59064393 0.70858758]
 [0.11067381 0.13596001 0.74064147 0.29661303]]


In [3]:
import numpy as np

def kron_A_X(A):
    N = A.shape[0]
    out = np.zeros((N*2, N*2),dtype=A.dtype)
    for i in range(N):
        for j in range(N):
            value = A[i,j]
            out[i*2: i*2 + 2,j*2:j*2 + 2] = np.array([[0, value], [value, 0]])
    return out


A = np.random.rand(3, 3)
result = kron_A_X(A)
print(result)


[[0.         0.34270731 0.         0.84177329 0.         0.58071937]
 [0.34270731 0.         0.84177329 0.         0.58071937 0.        ]
 [0.         0.89888455 0.         0.51334885 0.         0.28378635]
 [0.89888455 0.         0.51334885 0.         0.28378635 0.        ]
 [0.         0.64201619 0.         0.64069361 0.         0.25695429]
 [0.64201619 0.         0.64069361 0.         0.25695429 0.        ]]


In [78]:
np.allclose(np.kron(A, np.array([[0, 1], [1, 0]])), result)

True

In [85]:
A = np.random.rand(300, 300)
B = np.array([[1,1],[1,1]])

In [91]:
for i in range(10000):
    theta = np.pi / 4
    cos_theta_2 = np.cos(theta / 2)
    sin_theta_2 = np.sin(theta / 2)
    i = 1j
    
    RX = np.array([
        [cos_theta_2, -i * sin_theta_2],
        [-i * sin_theta_2, cos_theta_2]
    ], dtype=complex)
    k = np.kron(A, RX)

In [89]:
import numpy as np




In [90]:
import tensor
for i in range(10000):
    theta = np.pi / 4
    k1 = kron_A_RX(A, theta)

In [92]:
np.allclose(k1,k)

True

In [72]:
k1[:2,:2]

array([[0.81997325+0.j, 0.0025444 +0.j],
       [0.90099698+0.j, 0.80737652+0.j]])

In [73]:
k[:2,:2]

array([[0.21601802+0.j, 0.54611329+0.j],
       [0.02745547+0.j, 0.13916608+0.j]])

In [56]:
np.allclose(k1, k)

False

In [64]:
import numpy as np

def kron_RX_A(theta, A):
    cos_theta_2 = np.cos(theta / 2)
    sin_theta_2 = np.sin(theta / 2)
    i = 1j
    
    RX = np.array([
        [cos_theta_2, -i * sin_theta_2],
        [-i * sin_theta_2, cos_theta_2]
    ], dtype=complex)
    
    m, n = A.shape
    out = np.zeros((2, m, 2, n), dtype=complex)
    
    out[0, :, 0, :] = cos_theta_2 * A
    out[0, :, 1, :] = -i * sin_theta_2 * A
    out[1, :, 0, :] = -i * sin_theta_2 * A
    out[1, :, 1, :] = cos_theta_2 * A
    
    out.shape = (2 * m, 2 * n)
    return out

# Example usage:
theta = np.pi / 4
A = np.array([[1, 1], [1, 1]])
result = kron_RX_A(theta, A)
print(np.round(result, 4))


[[0.9239+0.j     0.9239+0.j     0.    -0.3827j 0.    -0.3827j]
 [0.9239+0.j     0.9239+0.j     0.    -0.3827j 0.    -0.3827j]
 [0.    -0.3827j 0.    -0.3827j 0.9239+0.j     0.9239+0.j    ]
 [0.    -0.3827j 0.    -0.3827j 0.9239+0.j     0.9239+0.j    ]]


In [61]:
np.allclose(

0.9238795325112867

In [7]:
from string import ascii_lowercase as letters
from functools import reduce

def mv_kron(matrices, y):
    '''
    Compute product of vector and Kronecker-structured matrix
    via brute-force enumeration of entire Kronecker matrix.
    '''
    A_kron = reduce(np.kron, matrices)
    return A_kron @ y.ravel()

def mv_einstein(matrices, y):
    '''
    Use Einstein summation convention to iteratively
    contract along secondary axes and implement Kronecker 
    matrix-vector product
    '''
    p = len(matrices)
    
    if p > 13:
        raise ValueError('There aren\'t enough letters in the alphabet for this operation :(')
    
    letter_pairs = [letters[2*i]+letters[2*i+1] for i in range(p)]
    matrix_string = ','.join(letter_pairs)
    vec_in_string, vec_out_string = [''.join(s) for s in zip(*letter_pairs)]
    string_spec = f'{matrix_string},{vec_in_string}->{vec_out_string}'
    
    return np.einsum(string_spec, *matrices, y, optimize='greedy').ravel(), string_spec

In [8]:
ein_times  = []
kron_times = []
dimensions = []
import time
for scale in [1, 2, 3, 4, 5, 6, 7, 8]:
    sizes = [2*scale, 4*scale, 8*scale]
    matrices = [np.random.randn(n,n) for n in sizes]
    matrices = [X@X.T for X in matrices]
    y = np.random.randn(*sizes)
    
    start_kron = time.perf_counter()
    mv_kron(matrices, y)
    end_kron = time.perf_counter() - start_kron
    
    start_ein = time.perf_counter()
    _, string_spec = mv_einstein(matrices, y)
    end_ein = time.perf_counter() - start_ein
    
    ein_times  += [end_ein]
    kron_times += [end_kron]
    dimensions += [y.size]
    print('Dimension:', str(y.size).ljust(5), f'Einstein time: {end_ein:.3f} s.', f'Naive time {end_kron:.3f}s.')


Dimension: 64    Einstein time: 0.000 s. Naive time 0.000s.
Dimension: 512   Einstein time: 0.000 s. Naive time 0.008s.
Dimension: 1728  Einstein time: 0.000 s. Naive time 0.008s.
Dimension: 4096  Einstein time: 0.000 s. Naive time 0.049s.
Dimension: 8000  Einstein time: 0.010 s. Naive time 0.172s.
Dimension: 13824 Einstein time: 0.020 s. Naive time 0.478s.
Dimension: 21952 Einstein time: 0.052 s. Naive time 1.247s.
Dimension: 32768 Einstein time: 0.014 s. Naive time 4.383s.
